In [ ]:
from rdhyee_utils.clipboard.macos import GeneralPasteboard, ptypes
pb = GeneralPasteboard()
data = pb.get_string()

data

In [ ]:
import sh

input_text = sh.pbpaste()
input_text

In [ ]:
import sh
import shlex
import json

from diff_match_patch import diff_match_patch

from IPython.display import display, HTML
from diff_match_patch import diff_match_patch

input_text = sh.pbpaste()

# r = json.loads(sh.llm(shlex.split('''-t copyedit ''')  + [input_text]  ))
r = sh.llm(["-t", "copyedit", input_text ] )
try:
    r = json.loads(r)
except json.JSONDecodeError:
    print (r)
    raise

output_text = r.get('edited_text')
comments = r.get('comments_and_questions')

print (output_text)
sh.pbcopy(_in=output_text)

for line in comments:
    print (line)



In [ ]:

dmp = diff_match_patch()
dmp.Diff_Timeout = 0 # unlimited

patches = dmp.patch_make(input_text, output_text)
diff_from_patches = dmp.patch_toText(patches)

diffs = dmp.diff_main(input_text, output_text)
diff_html = dmp.diff_prettyHtml(diffs)

display(HTML(diff_html))

# r = sh.llm("number of perfect numbers between 1 and 1000")
# r

In [ ]:
print(diff_from_patches)

In [ ]:
# https://llm.datasette.io/en/stable/python-api.html

import llm

model = llm.get_model("gpt-4o-mini")

# use template copyedit
# llm -t copyedit "Life is it's own reward."
response = model.prompt(prompt="Life is it's own reward.") #  template="copyedit")
# response = model.prompt("Five surprising names for a pet pelican")
# print(response.text())
for chunk in response:
    print(chunk, end="")

In [ ]:
import llm

model = llm.get_model("gpt-4o-mini")

# Load the template
template = llm.cli.load_template("copyedit")

# Use the template with your prompt
response = model.prompt(template.format(text="Life is it's own reward."))

# Print the response
for chunk in response:
    print(chunk, end="")

# using OpenAI to translate Moby Dick

In [ ]:
from pathlib import Path as P
from lxml import etree
from itertools import islice

import pandas as pd
import numpy as np
from pandas import DataFrame, Series


In [ ]:
# read in the XML in .bike and parse using lxml 
# get all the p text
# https://lxml.de/parsing.html


def extract_p_text(file_path):
    # Ensure the file path is a Path object
    if not isinstance(file_path, P):
        file_path = P(file_path)

    # Read the xml file
    xml_data = file_path.read_bytes()

    # Parse the XML data
    root = etree.fromstring(xml_data)

    # Find all 'p' elements
    p_elements = root.findall('.//p')

    # Extract and return the text from the 'p' elements

    for p in p_elements:
        if p.text is not None:
            yield (p.text)

# Usage
file_path = P.home() / "Downloads" / "moby_dick.bike" 
p_text_list = extract_p_text(file_path)

for i, text in islice(enumerate(p_text_list), 10):
    print(f"Paragraph {i+1}: {text}")


In [ ]:
file_path = P.home() / "Downloads" / "moby_dick.bike" 

paragraphs = list(extract_p_text(file_path))

In [ ]:
s = Series(paragraphs)
max(s.apply(len))

In [ ]:
import subprocess

openai_api_ref = "op://Private/g2mohpraf2d67jktlsoljc43si/Section_caj3qfdvfnkqfxn63jhjpydb7e/api key"
claude_api_ref = "op://Private/Claude key-20240304/credential"


def run_command(cmd):
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True)
    return result.stdout + result.stderr

def op_read(api_ref):
    cmd = f'op read "{api_ref}"'
    api_key = run_command(cmd).strip()
    return api_key



In [ ]:
import os
import openai

# Assuming the OpenAI API key is set as an environment variable:
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

# Ensure the API key is available
if api_key is None:
    raise ValueError("API key for OpenAI is not set")
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You will be provided with a sentence in English, and your task is to translate it into French.\n\n"
    },
    {
      "role": "user",
      "content": "Hello world"
    },
    {
      "role": "assistant",
      "content": "Bonjour le monde"
    },
       {
      "role": "user",
      "content": "Je speak French fairly well."
    },
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

response

In [ ]:
import json

# Assuming 'response' is the variable holding the API response.
# You might need to use response.json() if you're using requests library to fetch the data.

# Convert the response to a JSON object if not already done
response_json = json.loads(response.json())


# Accessing the list of messages in the response
message = response_json['choices'][0]['message']

    
role = message['role']
content = message['content']
print(f"Role: {role}, Content: {content}")

# To extract specific translations or interactions, you could filter by role
# For example, to print only the assistant's translations:
print("\nTranslations by the assistant:")
if message['role'] == 'assistant':
    print(message['content'])

# In case you want to handle more complex structures or search for specific dialog turns
# You might implement additional parsing logic here


In [ ]:

# Print the response text
print(response.choices[0].text.strip())


In [ ]:
response

In [ ]:
for i in range(10):
    print(f"\r{i}", end="")

In [ ]:
START = 1001
STOP = 3000

responses = []
for (i,p) in enumerate(paragraphs[START:STOP]):
    
    print(f"\r{i}", end="")
    messages = [
    {
      "role": "system",
      "content": "You will be provided with a sentence in English, and your task is to translate it into French."
    },
    {
      "role": "user",
      "content": p 
    },
    {
      "role": "assistant",
      "content": "" 
    }
  ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages= messages,
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    responses.append(response)

In [ ]:
responses

In [ ]:
for response in responses:
    print(response['choices'][0]['message']['content'])


In [ ]:
response

In [ ]:
# use openai to return the list of models, and their descriptions and costs
# https://beta.openai.com/docs/api-reference/list-engines


models = openai.Model.list()

for model in models['data']:
    print(f"Model ID: {model['id']}, Model Name: {model['object']}")




In [ ]:
import requests
from lxml import html

# URL of the page you want to scrape
url = 'https://openai.com/pricing'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page using lxml
tree = html.fromstring(response.content)


In [ ]:

# Define the XPath query to find the elements you're interested in
# This is a generic XPath and should be customized for your specific needs
# For example, to extract titles: '//h1/text()' or to extract prices: '//span[@class="price"]/text()'
xpath_query = '//*[@id="gpt-4-turbo"]/div/div/div[1]/div/div[1]/h3'

# Use the XPath query to extract the desired information
extracted_data = tree.xpath(xpath_query)

# Print the extracted data
for data in extracted_data:
    print(data.text)

In [ ]:
# https://chat.openai.com/share/d06bc114-e94d-48be-bf96-b9f73a2e9ea6

import requests
from bs4 import BeautifulSoup

# Define your local data to compare against
local_data = {
    "text_models": {
        "gpt-4": {
            "prompt_tokens": {
                "8k_context": "$0.03 / 1K tokens",
                "32k_context": "$0.06 / 1K tokens"
            },
            "sampled_tokens": {
                "8k_context": "$0.06 / 1K tokens",
                "32k_context": "$0.12 / 1K tokens"
            }
        },
        # Add other local data similarly
    },
    # Define other model types similarly
}

def fetch_pricing():
    url = "https://openai.com/pricing"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Assuming data is stored in a table or a specific div - adjust as necessary
    data_div = soup.find_all("div", class_="pricing-table")
    fetched_data = {}

    for div in data_div:
        model_name = div.find("h3").text
        price_info = div.find_all("p")  # Assuming price is in <p> tags
        prices = {p.text.split(':')[0].strip(): p.text.split(':')[1].strip() for p in price_info}
        fetched_data[model_name] = prices

    return fetched_data

def compare_data(fetched_data):
    differences = {}
    for category in local_data:
        if category in fetched_data:
            for model, details in local_data[category].items():
                if model in fetched_data[category]:
                    for detail, value in details.items():
                        if detail in fetched_data[category][model]:
                            fetched_value = fetched_data[category][model][detail]
                            if fetched_value != value:
                                differences[model + ' ' + detail] = (value, fetched_value)
    return differences

# Fetch the latest pricing data from OpenAI's website
fetched_pricing_data = fetch_pricing()

# Compare the fetched data with local data
differences = compare_data(fetched_pricing_data)

# Output the differences
if differences:
    print("There are differences in the pricing data:")
    for diff in differences:
        print(f"{diff}: Local = {differences[diff][0]}, Fetched = {differences[diff][1]}")
else:
    print("The local data matches the fetched data.")



# Difflib

In [ ]:
import re

def split_with_whitespace(s):
    return re.split(r'(\s+)', s)

# Example usage:
s = 'Hello   World \n This is a test'
print(split_with_whitespace(s))

In [ ]:
# hello world code for using difflib to compare two text blobs and show the diff in HTML format
# https://stackoverflow.com/questions/47694421/difflib-html-diff-output

import re
import difflib

from IPython.display import HTML

def tokenize(s):
    """Tokenize a string into words, punctuation, and whitespace.
    """
    return re.findall(r'\w+|\s+|\W', s)
    # return re.split(r'(\s+|\W+)', s)


def compare_by_chars(text1, text2):
    words1 = text1
    words2 = text2
    diff = difflib.ndiff(words1, words2)
    return diff

def compare_by_words(text1, text2):
    lines1 = tokenize(text1)
    lines2 = tokenize(text2)
    diff = difflib.ndiff(lines1, lines2, linejunk=lambda s: False)
    return diff


def ndiff_as_html(diff):
    html = []
    for line in diff:
        if line.startswith('- '):
            html.append(f'<span style="background-color: #ffcccc;">{line[2:]}</span>')
        elif line.startswith('+ '):
            html.append(f'<span style="background-color: #ccffcc;">{line[2:]}</span>')
        elif line.startswith('? '):
            html.append(f'<span style="background-color: #ffff99;">{line[2:]}</span>')
        elif line.startswith('  '):
            html.append(f'<span>{line[2:]}</span>')
    return ''.join(html)


# Example usage
text1 = """
I'm going to work from the bottom up and top down and from sideways too. So one idea for a specific project that I want to build very soon is to hook up a writing environment like Microsoft Word which has track changes and allow it to let me query something like chat gpt API to have it lightly copy at it my text and instead of just replacing all my text wholesale with the new revision to rather insert it and display the suggested changes as granular changes I can accept or reject. That is, I want to use the track changes to let me have much more fine grain control over my interactions with a large language model.
""".strip()

text2 = """
I'm going to work from the bottom up and top down, and from sideways too. So one idea for a specific project that I want to build very soon is to hook up a writing environment like Microsoft Word, which has track changes, and allow it to let me query something like the ChatGPT API to have it lightly copy-edit my text. Instead of just replacing all my text wholesale with a new revision, I want it to insert the suggested changes and display them as granular changes I can accept or reject. That is, I want to use the track changes feature to give me much more fine-grained control over my interactions with a large language model.
""".strip()

# diff = compare_by_chars(text1, text2)
diff = compare_by_words(text1, text2)

HTML(ndiff_as_html(diff))

# HTML(difflib.HtmlDiff().make_table(tokenize(text1), tokenize(text2), context=True, numlines=3))


In [ ]:
dmp = diff_match_patch()
dmp.Diff_Timeout = 0 # unlimited

diffs = dmp.diff_main(text1, text2)
diff_html = dmp.diff_prettyHtml(diffs)

display(HTML(diff_html))

In [ ]:
from diff_match_patch import diff_match_patch

dmp = diff_match_patch()
dmp.Diff_Timeout = 0 # unlimited

patches = dmp.patch_make(text1, text2)
diff = dmp.patch_toText(patches)



In [ ]:
from IPython.display import display, HTML
from diff_match_patch import diff_match_patch

text1="""
1.2.1 
en raison des dommages corporels, matériels et immatériels consécutifs ou 
non consécutifs subis par des tiers résultant d’une pollution imputable aux 
activités garanties de l’assuré que ce soient celles réalisées sur les sites 
couverts ou au titre des opérations couvertes. 
 

 
 
201811 
cg_chubb protection environnementale_2017 
4 
 
sont également inclus dans cette garantie, en tant qu’ils sont causés par une pollution imputable aux activités 
garanties de l’assuré que ce soient celles réalisées sur les sites couverts ou au titre des opérations couvertes: 
 
les frais de dépollution et de remise en etat dès lors que ces frais sont engagés pour réparer des 
dommages matériels subis par des tiers ; 
 
les conséquences pécuniaires des dommages subis par des biens confiés à l’assuré ou par des biens 
appartenant aux préposés de l’assuré dans la limite fixée aux conditions particulières. 
1.2.2 en raison des frais de réparation du préjudice ecologique, des dépenses de prévention du préjudice ecologique 
et du coût des mesures raisonnables relatifs à un préjudice écologique résultant d’une pollution imputable aux 
activités garanties de l’assuré que ce soit celles réalisées sur les sites couverts ou au titre des opérations 
couvertes. pour les seules atteintes à la biodiversité, la présente garantie 1.2.2. couvre également les frais de 
réparation du préjudice ecologique, les dépenses de prévention du préjudice ecologique et le coût des mesures 
raisonnables relatifs à un préjudice écologique afférents à ces atteintes à la biodiversité même lorsqu’elles n’ont 
pas pour origine une pollution dès lors qu’elles résultent d’un fait fortuit, imprévu et involontaire et sont 
imputables aux activités garanties de l’assuré que ce soit celles réalisées sur les sites couverts ou au titre des 
opérations couvertes . 
sont également inclus dans la garantie 1.2 les frais de défense et recours tels que définis à l’article 2.20. 
1.3 
garantie responsabilité environnementale  
les frais de réparation du dommage environnemental incombant à l’assuré au titre de sa responsabilité 
environnementale lorsque, conformément à l’article l. 162-11 du code de l’environnement ou aux textes équivalents 
adoptés par chacun des etats membres de l’union européenne pour transposer la directive européenne ce 2004/35, 
l’assuré se voit prescrire par une autorité publique compétente de mettre en œuvre des mesures destinées à réparer un 
dommage environnemental imputable aux activités garanties de l’assuré réalisées sur les sites couverts. 
les frais de réparation du dommage environnemental incombant à l’assuré au titre de sa responsabilité 
environnementale lorsque, conformément à l’article l. 162-11 du code de l’environnement ou aux textes équivalents 
adoptés par chacun des etats membres de l’union européenne pour transposer la directive européenne ce 2004/35, 
l’assuré se voit prescrire par une autorité publique compétente de mettre en œuvre des mesures destinées à réparer un 
dommage environnemental imputable aux activités garanties de l’assuré réalisées au titre des opérations couvertes. 
les frais de réparation du dommage environnemental incombant à l’assuré au titre de sa responsabilité 
environnementale lorsque, conformément à l’article l. 162-11 du code de l’environnement ou aux textes équivalents 
adoptés par les etats membres de l’union européenne pour transposer la directive européenne ce 2004/35, l’assuré se 
voit prescrire par l’autorité publique compétente de mettre en œuvre des mesures destinées à réparer un dommage 
environnemental imputable aux activités garanties de l’assuré provenant des produits couverts. 
les sommes exposées par l'etat au titre des frais de réparation du dommage environnemental conformément aux 
articles l. 162-14 et suivants du code de l’environnement ou aux textes équivalents adoptés par les etats membres de 
l’union européenne pour transposer la directive européenne ce 2004/35 et dont le remboursement est réclamé à 
l'assuré au titre de sa responsabilité environnementale dès lors que le dommage environnemental résulte des activités 
garanties de l’assuré quelles soient réalisées sur les sites couverts, au titre des opérations couvertes ou provenant des 
produits couverts. 
la présente garantie couvre également les frais de réparation du dommage environnemental relevant de la 
responsabilité environnementale de l’assuré même lorsqu’ils n’ont pas pour origine une pollution. dès lors qu’elles sont 
imputables aux activités garanties de l’assuré que ce soit celles réalisées sur les sites couverts ou au titre des opérations 
couvertes . 
sont également inclus dans la garantie 1.3 les frais de défense et recours tels que définis à l’article 2.20. 
"""


text2="""
1.2.1 
en raison des dommages corporels, matériels et immatériels consécutifs ou 
non consécutifs subis par des tiers résultant d’une pollution imputable aux 
activités garanties de l’assuré. sont également inclus dans cette garantie : 
• 
les frais de dépollution et de remise en état engagés pour réparer des 
dommages matériels subis par les tiers ; 
• 
les conséquences pécuniaires des dommages subis par des biens confiés à 
l’assuré ou par des biens appartenant aux préposés de l’assuré dans la 
limite fixée aux conditions particulières, 
dès lors qu’ils sont causés par une pollution imputable aux activités garanties 
de l’assuré et font l’objet d’une réclamation du tiers.  
1.2.2 en raison des préjudices écologiques résultant d’une pollution imputable aux 
activités garanties de l’assuré. pour les seules atteintes à la biodiversité, la 
présente garantie 1.2.2. couvre également les préjudices écologiques même 
lorsqu’elles n’ont pas pour origine une pollution dès lors qu’elles résultent d’un 
fait fortuit, imprévu et involontaire et sont imputables aux activités garanties 
de l’assuré. 
sont inclus dans la garantie 1.2 les frais de défense et recours tels que définis à 
l’article 2.20. 
conditions générales 

 
202306 
cg_chubb protection environnementale_2023 
4 
 
1.3 
garantie responsabilité environnementale  
les frais de réparation du dommage environnemental incombant à l’assuré au titre de sa responsabilité 
environnementale lorsque, conformément à l’article l. 162-11 du code de l’environnement ou aux textes équivalents 
adoptés par chacun des etats membres de l’union européenne pour transposer la directive européenne ce 2004/35, 
l’assuré se voit prescrire par une autorité publique compétente de mettre en œuvre des mesures destinées à réparer un 
dommage environnemental imputable aux activités garanties de l’assuré. 
les sommes exposées par l'etat au titre des frais de réparation du dommage environnemental conformément aux 
articles l. 162-14 et suivants du code de l’environnement ou aux textes équivalents adoptés par les etats membres de 
l’union européenne pour transposer la directive européenne ce 2004/35 et dont le remboursement est réclamé à 
l'assuré au titre de sa responsabilité environnementale dès lors que le dommage environnemental résulte des activités 
garanties de l’assuré. 
la présente garantie couvre également les frais de réparation du dommage environnemental relevant de la 
responsabilité environnementale de l’assuré même lorsqu’ils n’ont pas pour origine une pollution dès lors qu’ils sont 
imputables aux activités garanties de l’assuré. 
sont également inclus dans la garantie 1.3 les frais de défense et recours tels que définis à l’article 2.20. 
"""

dmp = diff_match_patch()
dmp.Diff_Timeout = 0 # unlimited

diffs = dmp.diff_main(text1, text2)
diff_html = dmp.diff_prettyHtml(diffs)

display(HTML(diff_html))


In [ ]:
import inspect


source_code = inspect.getsource(difflib.IS_LINE_JUNK)
print(source_code)


In [ ]:
import difflib

original = "The quick brown fox."
edited = "The quick brown foxx."

diff = difflib.ndiff(original, edited)
print('\n'.join(diff))